# Simulation programming project

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
data = pd.read_csv('DemandAndFulfillmentLog.csv')

In [3]:
df = data.copy(deep=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078 entries, 0 to 5077
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   RequestedDateTime  5078 non-null   object
 1   Rig                5078 non-null   object
 2   Material           5078 non-null   object
 3   QtyRequested       5078 non-null   int64 
 4   ReceivedDateTime   4837 non-null   object
 5   QtyReceived        5078 non-null   int64 
 6   Vessel             4879 non-null   object
 7   OriginPort         4879 non-null   object
dtypes: int64(2), object(6)
memory usage: 317.5+ KB


In [5]:
df.describe(include=['O'])

,RequestedDateTime,Rig,Material,ReceivedDateTime,Vessel,OriginPort
count,5078,5078,5078,4837,4879,4879
unique,373,4,6,4817,10,2
top,12/4/2021 0:00,BetaRig,DryBulk,7/24/2021 22:59,150C,HamburgPort
freq,19,1492,900,2,622,2463


In [6]:
df.describe(exclude=['O'])

,QtyRequested,QtyReceived
count,5078.000000,5078.000000
mean,1484.983655,1410.684915
std,1972.360420,1954.255474
min,1.000000,0.000000
25%,12.000000,9.000000
50%,60.000000,32.000000
75%,2500.000000,2400.000000
max,9800.000000,9800.000000


In [7]:
# format datetime

for col in ['RequestedDateTime', 'ReceivedDateTime'] :
    datetime = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
    df[col] = datetime
df

,RequestedDateTime,Rig,Material,QtyRequested,ReceivedDateTime,QtyReceived,Vessel,OriginPort
0,2021-01-01 08:00:00,AlphaRig,DeckCargo,5,2021-01-06 05:14:00,5,150A,RotterdamPort
1,2021-01-01 08:00:00,BetaRig,DeckCargo,7,2021-01-04 08:49:00,7,150B,RotterdamPort
2,2021-01-01 08:00:00,BetaRig,LiquidBulk,2000,2021-01-04 09:59:00,2000,150B,RotterdamPort
3,2021-01-01 08:00:00,BetaRig,Fuel,7700,2021-01-04 10:39:00,7700,150B,RotterdamPort
4,2021-01-01 08:00:00,BetaRig,DryBulk,2300,2021-01-04 14:30:00,2300,150B,RotterdamPort
...,...,...,...,...,...,...,...,...
5073,2021-12-31 08:00:00,CharlieRig,Fuel,1400,NaT,0,NaN,NaN
5074,2021-12-31 08:00:00,CharlieRig,Casing,9,NaT,0,NaN,NaN
5075,2021-12-31 08:00:00,CharlieRig,DryBulk,3100,NaT,0,NaN,NaN
5076,2021-12-31 08:00:00,DeltaRig,Casing,20,NaT,0,NaN,NaN


# Feature Engineering
In this cell, multiple columns are created to help with the analysis stage  
`TimeDiff` *datetime* : Total time spend between time of request and time of delivery  
`IsLate` *boolean* : True for "late delivery" (at least 5 days adter requested)  
`TimeLate` *datetime* : `TimeDiff` minus 5 days, clipped out negative values for in-time delivery  
`LatePenalty` *float* : Cost penalty for late delivery ($10,000 per hour)  
`QtyLost` *int* : Amount of cargo lost berfore delivery   
`Trip` *string* : Information about each delivery trip, comprise of OriginPort-Vessel-Rig

In [12]:
# feature engineering

df['TimeDiff'] = df['ReceivedDateTime'] - df['RequestedDateTime']
df['IsLate'] = df['TimeDiff'] > pd.Timedelta('5 days')
df['TimeLate'] = (df['TimeDiff'] - pd.Timedelta('5 days')).clip(lower='00:00:00')
# late penalty of $10000 per hour if IsLate = true (a fraction of an hour late is calculated as one full hour late)
df['LatePenalty'] = df['IsLate'] * (df['TimeLate'].dt.total_seconds() // 3600 + 1) * 10000
df['QtyLost'] = df['IsLate'] * (df['QtyRequested'] - df['QtyReceived'])
df['Trip'] = df['OriginPort'] + '-' + df['Vessel'] + '-' + df['Rig']

In [13]:
df.to_csv('DemandAndFulfillmentLog_formatted.csv', index=False)

# Missing data

In [16]:
df[df['ReceivedDateTime'].isnull() == True]

,RequestedDateTime,Rig,Material,QtyRequested,ReceivedDateTime,QtyReceived,Vessel,OriginPort,TimeDiff,IsLate,TimeLate,LatePenalty,QtyLost,Trip
4569,2021-11-25 00:00:00,DeltaRig,Casing,9,NaT,0,150I,HamburgPort,NaT,False,NaT,NaN,0,HamburgPort-150I-DeltaRig
4570,2021-11-25 00:00:00,DeltaRig,DryBulk,2600,NaT,0,150I,HamburgPort,NaT,False,NaT,NaN,0,HamburgPort-150I-DeltaRig
4580,2021-11-26 00:00:00,CharlieRig,DryBulk,2600,NaT,0,150G,HamburgPort,NaT,False,NaT,NaN,0,HamburgPort-150G-CharlieRig
4581,2021-11-26 00:00:00,CharlieRig,Pipe,12,NaT,0,150G,HamburgPort,NaT,False,NaT,NaN,0,HamburgPort-150G-CharlieRig
4583,2021-11-26 00:00:00,DeltaRig,LiquidBulk,2700,NaT,0,150I,HamburgPort,NaT,False,NaT,NaN,0,HamburgPort-150I-DeltaRig
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5073,2021-12-31 08:00:00,CharlieRig,Fuel,1400,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN
5074,2021-12-31 08:00:00,CharlieRig,Casing,9,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN
5075,2021-12-31 08:00:00,CharlieRig,DryBulk,3100,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN
5076,2021-12-31 08:00:00,DeltaRig,Casing,20,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN


There are 241 rows without ReceivedDateTime data and most are in the last months of the year  
some rows have Vessel and OriginPort data, which probably means that the order has not been delivered to the destination rig but the order has been assigned  
the other rows do not have Vessel and OriginPort data, which probably means that the order has not been assigned to any delivery attempt  

# Analysis
## Vessel type
There is a total of ten vessel  
Only vessel type **150** is currently being used by the company

In [17]:
df['Vessel'].value_counts(dropna=False)

150C    622
150A    621
150D    608
150B    565
150I    450
150F    432
150E    427
150G    403
150J    379
150H    372
NaN     199
Name: Vessel, dtype: int64

## Boat trips
from crosstab analysis, Vessel 150A, 150B, 150CC, 150D is only being used for delivery between RotterdamPort and AlphaRig, BetaRig  
in contrast, Vessel 150E, 150F, 150G, 150H, 150I, 150J is only being used for delivery between HamburgPort and CharlieRig, DeltaRig

In [34]:
pd.crosstab(index=df['Rig'], columns=df['Vessel'])

Vessel,150A,150B,150C,150D,150E,150F,150G,150H,150I,150J
Rig,,,,,,,,,,
AlphaRig,227,201,270,230,0,0,0,0,0,0
BetaRig,394,364,352,378,0,0,0,0,0,0
CharlieRig,0,0,0,0,240,259,229,123,273,188
DeltaRig,0,0,0,0,187,173,174,249,177,191


In [35]:
pd.crosstab(index=df['OriginPort'], columns=df['Vessel'])

Vessel,150A,150B,150C,150D,150E,150F,150G,150H,150I,150J
OriginPort,,,,,,,,,,
HamburgPort,0,0,0,0,427,432,403,372,450,379
RotterdamPort,621,565,622,608,0,0,0,0,0,0


In [30]:
grouped_trips = df.sort_values(by=['Trip', 'ReceivedDateTime'], ascending=[True, True])
grouped_trips

,RequestedDateTime,Rig,Material,QtyRequested,ReceivedDateTime,QtyReceived,Vessel,OriginPort,TimeDiff,IsLate,TimeLate,LatePenalty,QtyLost,Trip
6,2021-01-01 08:00:00,CharlieRig,LiquidBulk,2800,2021-01-06 10:46:00,2800,150E,HamburgPort,5 days 02:46:00,True,0 days 02:46:00,30000.0,0,HamburgPort-150E-CharlieRig
17,2021-01-02 08:00:00,CharlieRig,DeckCargo,5,2021-01-06 11:42:00,5,150E,HamburgPort,4 days 03:42:00,False,0 days 00:00:00,0.0,0,HamburgPort-150E-CharlieRig
18,2021-01-02 08:00:00,CharlieRig,LiquidBulk,3100,2021-01-06 12:32:00,3100,150E,HamburgPort,4 days 04:32:00,False,0 days 00:00:00,0.0,0,HamburgPort-150E-CharlieRig
19,2021-01-02 08:00:00,CharlieRig,Fuel,1900,2021-01-06 13:34:00,1900,150E,HamburgPort,4 days 05:34:00,False,0 days 00:00:00,0.0,0,HamburgPort-150E-CharlieRig
20,2021-01-02 08:00:00,CharlieRig,Casing,9,2021-01-06 14:31:00,9,150E,HamburgPort,4 days 06:31:00,False,0 days 00:00:00,0.0,0,HamburgPort-150E-CharlieRig
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5073,2021-12-31 08:00:00,CharlieRig,Fuel,1400,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN
5074,2021-12-31 08:00:00,CharlieRig,Casing,9,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN
5075,2021-12-31 08:00:00,CharlieRig,DryBulk,3100,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN
5076,2021-12-31 08:00:00,DeltaRig,Casing,20,NaT,0,NaN,NaN,NaT,False,NaT,NaN,0,NaN


In [31]:
grouped_trips.to_csv('DemandAndFulfillmentLog_grouped.csv', index=False)